<a href="https://colab.research.google.com/github/kartk/ML/blob/main/Decision_Trees_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
import numpy as np
from typing import List, Tuple

# *Creating Data*

1)  For a given value of k, m, (number of features, number of data points), write a function to generate a trainingdata set based on the above scheme.

In [39]:
def datagenerator(k:int, m:int)->List[Tuple[List[int], int]]:
  """
  Input:
    k: number of features
    m: number of data points
  Returns:
    Numpy array of size (m,k)
  """
  data = []
  w_den = sum([np.power(0.9, i) for i in range(2, k+1)])
  for i in range(m):
    X = [np.random.choice([1,0], p=[0.5, 0.5])]
    for j in range(1,k):
      X.append(np.random.choice([X[j-1], 1-X[j-1]], p = [0.75, 0.25]))
    Y = X[1] if (sum([ np.power(0.9, j)/w_den * X_j for j, X_j in enumerate(X[1:])]) >= 0.5) else (1 - X[1])
    data.append((X,Y))
  return data



In [40]:
data = datagenerator(k=4, m =30)

In [ ]:
def fit(data:List[Tuple[List[int]]]):
  
